In [ ]:
import example_robot_data
import pinocchio as pin
import numpy as np
from pinocchio.visualize import MeshcatVisualizer

In [ ]:
# load robot model
z1 = example_robot_data.load("z1")

viz = MeshcatVisualizer(z1.model, z1.collision_model, z1.visual_model)
viz.initViewer(loadModel=True, open=True)
# Load the robot in the viewer.
viz.loadViewerModel()
viz.displayVisuals(True)

In [ ]:
%%html
<iframe src="http://jupyter.lar.upatras.gr:7000/static/" width="640" height="480"></iframe>

In [ ]:
q = z1.q0
viz.display(q)

In [ ]:
# function to compute FK for all frames/joints
def fk_all(model, data, q, v = None):
    if v is not None:
        pin.forwardKinematics(model, data, q, v) # FK and Forward Velocities
    else:
        pin.forwardKinematics(model, data, q) # FK
    pin.updateFramePlacements(model, data) # Update frames

In [ ]:
def simu_step(model, data, q, v, v_target, dt):
    fk_all(model, data, q, v)
    a_desired = (v_target - v) / dt
    tau = pin.rnea(model, data, q, v, a_desired)
    tau = np.maximum(-model.effortLimit, np.minimum(tau, model.effortLimit))
    a = pin.aba(model, data, q, v, tau)
    vn = np.maximum(-model.velocityLimit, np.minimum(v + a * dt, model.velocityLimit))
    return pin.integrate(model, q, vn * dt), vn

In [ ]:
model, data = z1.model, z1.model.createData()

# Let's create a random target
nq = model.nq # dim of joints
target = pin.randomConfiguration(model)

# Controller parameters
Kp = 1.
Kd = 0.1
Ki = 0.01

q = z1.q0.copy()
v = np.zeros_like(q)

sum_error = 0.
prev_error = target - q
dt = 0.01

viz.display(q)

In [ ]:
%%html
<iframe src="http://jupyter.lar.upatras.gr:7000/static/" width="640" height="480"></iframe>

In [ ]:
K = round(20. / dt) + 1
for k in range(K):
    # Let's compute the commands
    error = target - q
    if ((k + 1) % 50 == 0) or k == 0:
        print(str(round((k+1)*dt, 2)) + ":", round(sum(np.square(error)), 2))
    derror = (error - prev_error) / dt
    prev_error = np.copy(error)
    sum_error += error * dt
    u = Kp * error + Kd * derror + Ki * sum_error
    # We set the commands
    q, v = simu_step(model, data, q, v, u, dt)
    # Visualize
    viz.display(q)